In [1]:
# Load dependencies
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.1e}'.format
import sys
sys.path.insert(0,'../../../../statistics_helper/')
from excel_utils import *

# Estimating the biomass of poultry
To estimate the biomass of poultry, we rely on data on global stocks of chickens, ducks, and turkeys from the Food and Agriculture Organization database FAOStat. We downloaded data from the domain Production/Live animals.
We combined data on the total stocks of each animal with estimates of the mean mass of each type of animal species (in kg) from [Dong et al.](http://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_10_Ch10_Livestock.pdf), Annex 10A.2, Tables 10A-4 to 10A-9.

Here are samples of the data:

In [2]:
# Load bird data
bird = pd.read_csv('FAOSTAT_data_bird.csv')
bird.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QA,Live Animals,5100,Africa,5112,Stocks,1057,Chickens,2014,2014,1000 Head,1809059,A,"Aggregate, may include official, semi-official..."
1,QA,Live Animals,5100,Africa,5112,Stocks,1068,Ducks,2014,2014,1000 Head,28539,A,"Aggregate, may include official, semi-official..."
2,QA,Live Animals,5100,Africa,5112,Stocks,1072,Geese and guinea fowls,2014,2014,1000 Head,25296,A,"Aggregate, may include official, semi-official..."
3,QA,Live Animals,5100,Africa,5112,Stocks,1083,"Pigeons, other birds",2014,2014,1000 Head,13384,A,"Aggregate, may include official, semi-official..."
4,QA,Live Animals,5100,Africa,5112,Stocks,1079,Turkeys,2014,2014,1000 Head,23658,A,"Aggregate, may include official, semi-official..."


In [3]:
# Load body mass data
body_mass = pd.read_csv('ipcc_animal_weight.csv')
body_mass.set_index('IPCC Area',inplace=True)
body_mass.head()

,Cattle - dairy,Cattle - non-dairy,Buffaloes,Swine - market,Swine - breeding,Chicken - Broilers,Chicken - Layers,Ducks,Turkeys,Sheep,Goats,Horses,Asses,Mules,Camels,Llamas
IPCC Area,,,,,,,,,,,,,,,,
Indian Subcontinent,275,110,295,28,28,9.0e-01,1.8e+00,2.7e+00,6.8e+00,2.8e+01,3.0e+01,238,130,130,217,217
Eastern Europe,550,391,380,50,180,9.0e-01,1.8e+00,2.7e+00,6.8e+00,4.8e+01,3.8e+01,377,130,130,217,217
Africa,275,173,380,28,28,9.0e-01,1.8e+00,2.7e+00,6.8e+00,2.8e+01,3.0e+01,238,130,130,217,217
Oceania,500,330,380,45,180,9.0e-01,1.8e+00,2.7e+00,6.8e+00,4.8e+01,3.8e+01,377,130,130,217,217
Western Europe,600,420,380,50,198,9.0e-01,1.8e+00,2.7e+00,6.8e+00,4.8e+01,3.8e+01,377,130,130,217,217


We pivot the stocks DataFrame to have a view of each kind of animal at each region:

In [4]:

# Replace NaN with zeros
bird.fillna(value=0,inplace=True)

bird_pivot = pd.pivot(bird.Area,bird.Item, bird.Value).astype(float)

# Replace NaN with zeros
bird_pivot.fillna(value=0,inplace=True)


bird_pivot

Item,Chickens,Ducks,Geese and guinea fowls,"Pigeons, other birds",Turkeys
Area,,,,,
Africa,1.8e+06,2.9e+04,2.5e+04,1.3e+04,2.4e+04
Americas,5.4e+06,2.8e+04,7.8e+02,0.0e+00,3.1e+05
Asia,1.2e+07,9.9e+05,2.8e+05,1.6e+04,1.5e+04
Eastern Europe,1.0e+06,5.4e+04,1.7e+04,0.0e+00,3.7e+04
Northern America,2.1e+06,9.3e+03,3.5e+02,0.0e+00,2.4e+05
Oceania,1.3e+05,1.4e+03,8.2e+01,0.0e+00,1.4e+03
Southern Asia,2.8e+06,7.6e+04,1.0e+03,0.0e+00,2.0e+03
Western Europe,4.5e+05,3.0e+04,8.6e+02,1.8e+03,3.3e+04


There is a difference between the body mass of a egg laying chicken to a non-egg laying chicken. We thus count seperately the egg laying chicken from the non-egg laying chicken. Data about the amount of egg laying chicken comes from the FAOStat domain Production - Livestock Primary.

In [5]:
# Load data about egg laying chicken
egg = pd.read_csv('FAOSTAT_data_eggs.csv')

# Set the index of the DataFrame to be the region so we can compare with the stocks data
egg.set_index('Area',inplace=True)

# Add a category of egg laying chicken
bird_pivot['Chicken - Layers'] = egg.Value

# Set the amount of non-egg laying chicken to be the total number minus the egg laying chicken
bird_pivot['Chickens'] -= egg.Value

# Rename the Chicken column name to Chicken - Broileers
bird_pivot.rename(columns={'Chickens': 'Chicken - Broilers'},inplace=True)

# Use only data for chicken, ducks and turkeys
birds = ['Chicken - Broilers','Chicken - Layers','Ducks','Turkeys']
bird_pivot_filt = bird_pivot[birds]
body_mass_filt = body_mass[birds]

bird_pivot_filt

Item,Chicken - Broilers,Chicken - Layers,Ducks,Turkeys
Area,,,,
Africa,1.3e+06,505203,2.9e+04,2.4e+04
Americas,4.3e+06,1149300,2.8e+04,3.1e+05
Asia,7.4e+06,4535174,9.9e+05,1.5e+04
Eastern Europe,5.8e+05,424370,5.4e+04,3.7e+04
Northern America,1.8e+06,398025,9.3e+03,2.4e+05
Oceania,1.0e+05,22334,1.4e+03,1.4e+03
Southern Asia,2.2e+06,660117,7.6e+04,2.0e+03
Western Europe,2.8e+05,168359,3.0e+04,3.3e+04


Data on the mass of animals is divided into different regions than the FAOStat data so we need preprocess the stocks DataFrame and merge it with the body mass data:

In [6]:
# Convert units
bird_pivot_filt *= 1e3

# Calculate the total number of animals in Latin America by subtracting values for Northern America from the total
# values for the Americas
bird_pivot_filt.loc['Americas'] -= bird_pivot_filt.loc['Northern America']

# Change name of Americas to Latin America
bird_pivot_filt.rename(index={'Americas': 'Latin America'},inplace=True)

# Calculate the total number of animals in Asia without the Indian Subcontinent by subtracting values for the Southern Asia 
# from the total values for the Asia
bird_pivot_filt.loc['Asia'] -= bird_pivot_filt.loc['Southern Asia']

# Change name of Southern Asia to Indian Subcontinent
bird_pivot_filt.rename(index={'Southern Asia': 'Indian Subcontinent'},inplace=True)

bird_pivot_filt

/home/yinon/.local/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/yinon/.local/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:13

Item,Chicken - Broilers,Chicken - Layers,Ducks,Turkeys
Area,,,,
Africa,1.3e+09,5.1e+08,2.9e+07,2.4e+07
Latin America,2.5e+09,7.5e+08,1.8e+07,6.9e+07
Asia,5.2e+09,3.9e+09,9.1e+08,1.3e+07
Eastern Europe,5.8e+08,4.2e+08,5.4e+07,3.7e+07
Northern America,1.8e+09,4.0e+08,9.3e+06,2.4e+08
Oceania,1.0e+08,2.2e+07,1.4e+06,1.4e+06
Indian Subcontinent,2.2e+09,6.6e+08,7.6e+07,2.0e+06
Western Europe,2.8e+08,1.7e+08,3.0e+07,3.3e+07


We now multiply the stocks of each animal type and for each region by the characteristic body weight of each animal:

In [7]:
wet_bird_biomass = (body_mass_filt*bird_pivot_filt)
wet_bird_biomass

,Chicken - Broilers,Chicken - Layers,Ducks,Turkeys
Africa,1.2e+09,9.1e+08,7.7e+07,1.6e+08
Asia,4.7e+09,7.0e+09,2.5e+09,8.6e+07
Eastern Europe,5.2e+08,7.6e+08,1.5e+08,2.5e+08
Indian Subcontinent,2.0e+09,1.2e+09,2.1e+08,1.4e+07
Latin America,2.3e+09,1.4e+09,5.0e+07,4.7e+08
Middle east,nan,nan,nan,nan
Northern America,1.6e+09,7.2e+08,2.5e+07,1.7e+09
Oceania,9.3e+07,4.0e+07,3.9e+06,9.4e+06
Western Europe,2.6e+08,3.0e+08,8.0e+07,2.3e+08


We sum over all regions and convert units from kg wet weight to Gt C carbon by assuming carbon is ≈15% of the wet weight (30% dry weight of wet weight and carbon is 50% of dry weight).

In [8]:
pd.options.display.float_format = '{:,.3f}'.format

# conversion factor from kg wet weight to Gt C
kg_to_gt_c = 1000*0.15/1e15
total_biomass = wet_bird_biomass.sum()*kg_to_gt_c
total_biomass

Chicken - Broilers   0.002
Chicken - Layers     0.002
Ducks                0.000
Turkeys              0.000
dtype: float64

In [9]:
best_estimate = total_biomass.sum()
print('Our estimate for the total biomass of poultry is ≈%.3f Gt C' % best_estimate)

Our estimate for the total biomass of poultry is ≈0.005 Gt C


In [10]:
update_MS_data(row='Biomass of poultry',values= best_estimate,path='../../../../results.xlsx')